In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://github.com/lukeNonyane1/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Activity3.02/fruits360.zip'

In [3]:
# Download the dataset
zip_dir = tf.keras.utils.get_file('fruits360', origin=file_url, extract=True)

In [4]:
zip_dir

'/Users/LNonyane/.keras/datasets/fruits360'

In [5]:
# Import the pathlib library
import pathlib

In [6]:
# Create a variable called path containing the full path to the fruits360_filtered directory using pathlib.Path(zip_dir).parent
path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'

In [7]:
print(path.parent)

/Users/LNonyane/.keras/datasets


In [8]:
# iterate and print each directory 
[x for x in path.iterdir() if x.is_dir()]

[PosixPath('/Users/LNonyane/.keras/datasets/fruits360_filtered/Test'),
 PosixPath('/Users/LNonyane/.keras/datasets/fruits360_filtered/Training')]

In [9]:
# Create two variables called train_dir and val_dir that take the full paths to the train and validation folders, respectively
train_dir = path / 'Training'
val_dir = path / 'Test'

In [10]:
# total_train and total_val
total_train, total_val = 11398, 4752 # values provided

In [11]:
# import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
# train_img_gen
train_img_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [13]:
val_img_gen = ImageDataGenerator(rescale=1./255)

In [14]:
# batch_size, img_heght, img_width, channel
batch_size, img_heght, img_width, channel = 16, 100, 100, 3

In [15]:
# create a data generator called train_data_gen using flow_from_directory
train_data_gen = train_img_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=train_dir,
                                                  target_size=(img_heght, img_width))

Found 11398 images belonging to 120 classes.


In [16]:
len(train_data_gen)

713

In [16]:
# create a data generator called val_data_gen using flow_from_directory
val_data_gen = val_img_gen.flow_from_directory(batch_size=batch_size,
                                                  directory=val_dir,
                                                  target_size=(img_heght, img_width))

Found 4752 images belonging to 120 classes.


In [17]:
# import required libraries
import numpy as np
from tensorflow.keras import layers

In [18]:
# set 8 as the seed for numpy and tensorflow using np.random.seed(8) and tf.random.set_seed(8)
np.random.seed(8)
tf.random.set_seed(8)

In [19]:
# import VGG16
from tensorflow.keras.applications import VGG16

In [20]:
# base_model
base_model = VGG16(input_shape=(img_heght,
                                img_width,
                                channel),
                   weights='imagenet',
                   include_top=False)

# freeze model so that weights will not be updated
base_model.trainable = False

2022-05-28 14:37:36.659721: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
# base_model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [22]:
# add fully connected layers to base_model
new_model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(120, activation='softmax')
])

In [23]:
# Only the prediction_layers' weights will be updated when the model is trained
optimizer = tf.keras.optimizers.Adam(0.001)
new_model.compile(loss='categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [24]:
# summary of new model
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 1000)              4609000   
                                                                 
 dense_1 (Dense)             (None, 120)               120120    
                                                                 
Total params: 19,443,808
Trainable params: 4,729,120
Non-trainable params: 14,714,688
_________________________________________________________________


In [25]:
# fit model and provide the train and validation data generators, epochs=5, and the validation steps
new_model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5


/var/folders/fm/b49y7nm968107m07lbt3cqsc0000gp/T/ipykernel_47437/4064329950.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  new_model.fit_generator(


712/712 [==============================] - 292s 409ms/step - loss: 2.0464 - accuracy: 0.5031 - val_loss: 1.0113 - val_accuracy: 0.7302
Epoch 2/5
712/712 [==============================] - 320s 450ms/step - loss: 0.6719 - accuracy: 0.8004 - val_loss: 0.6865 - val_accuracy: 0.8131
Epoch 3/5
712/712 [==============================] - 323s 454ms/step - loss: 0.4499 - accuracy: 0.8629 - val_loss: 0.6164 - val_accuracy: 0.8207
Epoch 4/5
712/712 [==============================] - 324s 455ms/step - loss: 0.3380 - accuracy: 0.8944 - val_loss: 0.4187 - val_accuracy: 0.8721
Epoch 5/5
712/712 [==============================] - 319s 449ms/step - loss: 0.2800 - accuracy: 0.9113 - val_loss: 0.3994 - val_accuracy: 0.8849
